In [ ]:
%%bash

pip -qqq install 'crewai[tools]'
pip -qqq install youtube-transcript-api
pip -qqq install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.3/463.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [ ]:
import os
from crewai import Agent
from google.colab import userdata
from crewai import Crew,Process
from crewai_tools import YoutubeChannelSearchTool
from crewai import Task

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
model_ID = userdata.get('GPT_MODEL')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
os.environ['OPENAI_MODEL_NAME']=model_ID

In [ ]:
from crewai_tools import YoutubeVideoSearchTool

In [ ]:
tool = YoutubeVideoSearchTool(youtube_video_url='https://www.youtube.com/watch?v=LfnrRPFhkuY')

In [ ]:
tool

YoutubeVideoSearchTool(name='Search a Youtube Video content', description="Search a Youtube Video content(search_query: 'string') - A tool that can be used to semantic search a query the https://www.youtube.com/watch?v=LfnrRPFhkuY Youtube Video content. search_query: 'Mandatory search query you want to use to search the Youtube Video content'", args_schema=<class 'crewai_tools.tools.youtube_video_search_tool.youtube_video_search_tool.FixedYoutubeVideoSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7a6375711ab0>, result_as_answer=False, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7a6372072f80>, summarize=False), config=None)

In [ ]:
topic_reaearch = Agent(
    role='Topic Research Agent',
    goal='Search for relevant information on the topic {topic} from the provided youtube video',
    verbose=True,
    memory=True,
    backstory='Expert in finding and analyzing relevant content from the youtube video, Specializing in AI, Data Science, Machine learning, and Generative AI',
    tools=[tool],
    allow_delegation=True
)

In [ ]:
linkedin_post_agent = Agent(
    role='Linkedin Post Creator',
    goal='Write a concise LinkedIn Post summary from the transcription provided by the Topic Researcher',
    verbose=True,
    memory=True,
    backstory='Expert in crafting engaging LinkedIn posts that summarize complex topics and include trending hashtags for maximum visibility',
    #tools=[tool], no tools required
    allow_delegation=False
)

In [ ]:
blog_writer = Agent(
    role='Blog Writer',
    goal='Write a comprehensive blog post from the transcription provided by the topic Researcher, covering all nessasary sections',
    verbose=True,
    memory=True,
    backstory='Expert in creating in-depth, well stuctured blog posts that explain technical concepts clearly and engage readers from introduction to conclus ion',
    #tools=[tool], no tools required
    allow_delegation=False
)

In [ ]:
research_task = Task(
    description = 'Identify and analyze video on the topic {topic} from the specified youtube video',
    expected_output='A complete word by word report on the most important points in the video for the topic {topic}',
    agent=topic_reaearch,
    tools=[tool]
)

In [ ]:
blog_writibg_task = Task(
    description = '''Write a comprehensive blog post based on the transcription provided by the Topic Reasearcher
                     article must include an introduction, step by step guides and conclusion. The overall content must be about 600 words long''',
    expected_output='A markdown-formatted of the blog',
    agent=blog_writer,
    output_file = 'blog-post.md'
)

In [ ]:
linkedin_post_task = Task(
    description = 'Write a linkedin post based on the transcription provided by the Topic Reasearcher, including relavant hashtags',
    expected_output='A markdown-formatted of the LinkedIn post',
    agent=linkedin_post_agent,
    output_file = 'linkedin-post.md'
)

In [ ]:
my_crew = Crew(
    agents=[topic_reaearch, blog_writer, linkedin_post_agent],
    tasks=[research_task, blog_writibg_task, linkedin_post_task],
    verbose=True,
    memory = True,
    cache = True,
    share_crew=True
)

In [ ]:
topic1 = 'LSTM'
result = my_crew.kickoff(inputs={'topic':topic1})
print('----------------------------------------')
print(result)

 [2024-09-09 16:26:32][DEBUG]: == Working Agent: Topic Research Agent
 [2024-09-09 16:26:32][INFO]: == Starting Task: Identify and analyze video on the topic LSTM from the specified youtube video


> Entering new CrewAgentExecutor chain...


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  5.93it/s]


Thought: I need to search the specified YouTube video for content related to the topic "LSTM."

Action: Search a Youtube Video content
Action Input: {"search_query": "LSTM"} 

Relevant Content:
Seen this movie called Memento or there is a bollywood movie called Ghajni? our basic RNNs are like the heroes of that movie they suffer from a short-term memory problem LSTM is a special version of RNN which solves the short-term memory problem and in this video I will explain LSTM in a very simple manner using real-life examples let's say you have NLP task for centers completion here in both the sentences based on what word appeared in the beginning your autocomplete sentence might be different for example for the first one I would say I need to take a loan whereas for the second one I would say I had to take a loan and this decision between need and head was made based on what appeared at the very beginning now looking at our traditional RNN and architecture if you have seen my previous video

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  9.30it/s]


I now can give a great answer

Final Answer:
```markdown
# Understanding LSTM: A Solution to Short-Term Memory Issues in RNNs

Have you ever watched the movie "Memento" or its Bollywood adaptation "Ghajni"? If you have, you'll recall how the protagonists struggle with short-term memory loss. Traditional Recurrent Neural Networks (RNNs) face a similar issue. They struggle to remember information from earlier in the sequence, which hampers their performance in tasks requiring long-term dependencies. Long Short-Term Memory (LSTM) networks, a special type of RNN, solve this problem. In this blog post, we'll break down LSTMs using real-life examples to make the concept easy to grasp.

## Introduction to LSTM

Imagine you have a natural language processing (NLP) task like sentence completion. The prediction of the next word in a sentence depends on the words that came before it. For instance, consider these two sentences:

1. "I need to take a loan."
2. "I had to take a loan."

The choice be